# **2153. The Number of Passengers in Each Bus II**

**Table: Buses**
``` sql
+--------------+------+
| Column Name  | Type |
+--------------+------+
| bus_id       | int  |
| arrival_time | int  |
| capacity     | int  |
+--------------+------+
```
bus_id contains unique values.

Each row of this table contains information about the arrival time of a bus at the LeetCode station and its capacity (the number of empty seats it has).

No two buses will arrive at the same time and all bus capacities will be positive integers.
 

**Table: Passengers**
``` sql
+--------------+------+
| Column Name  | Type |
+--------------+------+
| passenger_id | int  |
| arrival_time | int  |
+--------------+------+
```
passenger_id contains unique values.

Each row of this table contains information about the arrival time of a passenger at the LeetCode station.
 

Buses and passengers arrive at the LeetCode station. If a bus arrives at the station at a time tbus and a passenger arrived at a time tpassenger where tpassenger <= tbus and the passenger did not catch any bus, the passenger will use that bus. In addition, each bus has a capacity. If at the moment the bus arrives at the station there are more passengers waiting than its capacity capacity, only capacity passengers will use the bus.

Write a solution to report the number of users that used each bus.

Return the result table ordered by bus_id in ascending order.

The result format is in the following example.

 

**Example 1:**

**Input:** 
**Buses table:**
``` sql
+--------+--------------+----------+
| bus_id | arrival_time | capacity |
+--------+--------------+----------+
| 1      | 2            | 1        |
| 2      | 4            | 10       |
| 3      | 7            | 2        |
+--------+--------------+----------+
```
**Passengers table:**
``` sql
+--------------+--------------+
| passenger_id | arrival_time |
+--------------+--------------+
| 11           | 1            |
| 12           | 1            |
| 13           | 5            |
| 14           | 6            |
| 15           | 7            |
+--------------+--------------+
```
**Output:** 
``` sql
+--------+----------------+
| bus_id | passengers_cnt |
+--------+----------------+
| 1      | 1              |
| 2      | 1              |
| 3      | 2              |
+--------+----------------+
```
**Explanation:** 
- Passenger 11 arrives at time 1.
- Passenger 12 arrives at time 1.
- Bus 1 arrives at time 2 and collects passenger 11 as it has one empty seat.

- Bus 2 arrives at time 4 and collects passenger 12 as it has ten empty seats.

- Passenger 12 arrives at time 5.
- Passenger 13 arrives at time 6.
- Passenger 14 arrives at time 7.
- Bus 3 arrives at time 7 and collects passengers 12 and 13 as it has two empty seats.

---
**Solution-1:**
``` sql
WITH OrderedBusArrivals AS (
  -- Joining buses with passengers who arrived on or before each bus's arrival.
  -- Counting the number of passengers eligible to board each bus.
  SELECT 
    bus_id, 
    b.arrival_time, 
    capacity, 
    COUNT(passenger_id) AS eligible_passengers 
  FROM 
    Buses b 
    LEFT JOIN Passengers p ON p.arrival_time <= b.arrival_time 
  WHERE 
    bus_id IS NOT NULL 
  GROUP BY 
    bus_id 
  ORDER BY 
    b.arrival_time
) 
SELECT 
  bus_id, 
  passengers_cnt 
FROM 
  (
    SELECT 
      bus_id, 
      capacity, 
      eligible_passengers, 
      -- Calculating the number of passengers that can board the bus.
      -- Limited by either the bus's capacity or the remaining passengers after previous buses.
      @boarded_passengers := LEAST(
        capacity, eligible_passengers - @accumulated_boarding
      ) AS passengers_cnt, 
      -- Updating the total number of passengers who have boarded buses so far.
      @accumulated_boarding := @accumulated_boarding + @boarded_passengers 
    FROM 
      OrderedBusArrivals, 
      (
        SELECT 
          @accumulated_boarding := 0, 
          @boarded_passengers := 0
      ) AS Initialization
  ) AS FinalResult 
ORDER BY 
  bus_id;

---
**Solution-2:**
``` sql
WITH RECURSIVE

    -- Adding a column to track the arrival time of the previous bus
    UpdatedBuses AS (
        SELECT
            B.bus_id,
            B.arrival_time,
            B.capacity,
            -- Use LAG to find the arrival time of the previous bus
            COALESCE(LAG(B.arrival_time) OVER (ORDER BY B.arrival_time), 0) AS previous_bus_arrival
        FROM Buses B
    ),

    -- Counting new passengers arriving between the current and previous bus
    PassengerArrivalCounts AS (
        SELECT
            B.bus_id,
            B.arrival_time,
            B.capacity,
            B.previous_bus_arrival,
            -- Counting passengers arriving after the previous bus and before this bus
            COUNT(P.passenger_id) AS new_passengers,
            ROW_NUMBER() OVER (ORDER BY B.arrival_time) AS bus_sequence_number
        FROM UpdatedBuses B
        LEFT JOIN Passengers P
            ON P.arrival_time <= B.arrival_time AND P.arrival_time > B.previous_bus_arrival
        GROUP BY B.bus_id, B.arrival_time, B.capacity
    ),

    -- Recursive CTE to calculate passengers boarded and remaining for each bus
    BusBoardingDetails AS (
        -- Base case: Processing the first bus
        SELECT
            bus_sequence_number,
            bus_id,
            -- Boarding passengers limited by bus capacity
            LEAST(capacity, new_passengers) AS passengers_boarded,
            -- Remaining passengers who couldn't board the bus
            (new_passengers - LEAST(capacity, new_passengers)) AS passengers_remaining
        FROM PassengerArrivalCounts
        WHERE bus_sequence_number = 1

        UNION ALL

        -- Recursive case: Processing subsequent buses
        SELECT
            PAC.bus_sequence_number,
            PAC.bus_id,
            -- Boarding passengers, considering remaining passengers from previous buses
            LEAST(PAC.capacity, PAC.new_passengers + REC.passengers_remaining) AS passengers_boarded,
            -- Calculating remaining passengers
            (PAC.new_passengers + REC.passengers_remaining) - LEAST(PAC.capacity, PAC.new_passengers + REC.passengers_remaining) AS passengers_remaining
        FROM
            BusBoardingDetails REC,
            PassengerArrivalCounts PAC
        WHERE
            PAC.bus_sequence_number = REC.bus_sequence_number + 1
    )

-- Selecting the final bus boarding details
SELECT
    bus_id,
    passengers_boarded AS passengers_cnt
FROM BusBoardingDetails
ORDER BY bus_id;

---
**Solution-3:**
``` sql
WITH RECURSIVE

    --Add a "previous bus time" column:
    upd_buses AS (
        SELECT
            B.bus_id,
            B.arrival_time,
            B.capacity,
            COALESCE(LAG(B.arrival_time) OVER (ORDER BY B.arrival_time), 0) AS prev_bus_time
        FROM Buses B
    ),

    -- For each bus, add a new_pax column showing how many new passengers
    -- arrived to the bus stop since the last bus
    running_totals AS (
        SELECT
            B.bus_id,
            B.arrival_time,
            B.capacity,
            B.prev_bus_time,
            COUNT(P.passenger_id) AS new_pax,
            ROW_NUMBER() OVER (ORDER BY B.arrival_time) AS row_num
        FROM upd_buses AS B

        LEFT JOIN Passengers P
        ON
            P.arrival_time <= B.arrival_time
            AND
            P.arrival_time > B.prev_bus_time

        GROUP BY B.bus_id, B.arrival_time, B.capacity
    ),

    -- Use a recursive CTE to build the final table row-by-row. "Boarded" is how many
    --people entered the bus, and "remaining" is how many didn't fit in.
    recursive_cte AS (
        # Base - select first row
        SELECT
            row_num,
            bus_id,
            LEAST(capacity, new_pax) AS boarded,
            (new_pax - LEAST(capacity, new_pax)) AS remaining
        FROM running_totals
        WHERE row_num = 1

        UNION ALL

        -- Create all other rows, by combining the next row from the `running_totals` table,
        --- and the previously built row from this CTE
        SELECT
            T.row_num,
            T.bus_id,
            LEAST(capacity, new_pax + remaining) AS boarded,
            (new_pax + remaining) - LEAST(capacity, new_pax + remaining) AS remaining
        FROM
            recursive_cte REC,
            running_totals T
        WHERE
            T.row_num = REC.row_num + 1
    )

-- The data is ready, just select the columns we need
SELECT
    bus_id,
    boarded AS passengers_cnt
FROM recursive_cte
ORDER BY bus_id

---

**Solution-4:**
``` sql
WITH TEMP AS (
SELECT bus_id, b.arrival_time, capacity, count(passenger_id) AS num
FROM Buses b  LEFT JOIN Passengers p  ON p.arrival_time <= b.arrival_time
WHERE bus_id is not NULL
GROUP BY bus_id
ORDER BY arrival_time
)

SELECT bus_id, passengers_cnt from (
SELECT bus_id, capacity, num,
      @passengers_cnt:=LEAST(capacity,num-@accum) as passengers_cnt, 
      @accum:=@accum+@passengers_cnt
FROM TEMP, (SELECT @accum:= 0, @passengers_cnt:=0) INIT) temp
ORDER BY bus_id

----
**Solution-5:**
``` sql
# Write your MySQL query statement below
WITH RECURSIVE t1 AS(
    SELECT b.*,ROW_NUMBER() OVER(ORDER BY arrival_time) 'rn', COUNT(passenger_id) 'acc'
    FROM Buses b LEFT JOIN Passengers p ON b.arrival_time>=p.arrival_time
    GROUP BY b.bus_id
),
t2 AS(
    SELECT bus_id,LEAST(capacity,acc) 'passengers_cnt_acc',rn
    FROM t1 
    WHERE rn=1
    UNION ALL
    SELECT t1.bus_id,LEAST(t1.capacity,t1.acc-t2.passengers_cnt_acc)+t2.passengers_cnt_acc,t2.rn+1
    FROM t1,t2 
    WHERE t1.rn=t2.rn+1
),
t3 AS(
    SELECT bus_id,passengers_cnt_acc-LAG(passengers_cnt_acc,1,0)OVER(ORDER BY rn)'passengers_cnt' 
    FROM t2 
)
# SELECT * FROM t2
SELECT * FROM t3
ORDER BY bus_id 
```
---